In [1]:
import pandas as pd
import numpy as np
import xlsxwriter as xw
import re
import warnings
warnings.filterwarnings('ignore')
import openpyxl as op
from openpyxl import Workbook, load_workbook
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font
import os
import glob

In [2]:
Year = "2023"
Month = "Jul"
Month_n = "07"
Date_n_ob_no_change = "32"

In [3]:
filter_range = ['18-Jul-2023', '19-Jul-2023']
filter_form = ['Form Visit', 'Form OB', 'Form', 'Form Team View']

In [4]:
input_loc = f"/home/dhrubakumardolai/Desktop/Python Test/Result/Revisit File/{Year}/{Month}/Form/RWM/Consolidated/"
output_loc = "/home/dhrubakumardolai/Desktop/Python Test/Rahul/Payout Remarks/Result/"
input_loc_ob = f"/home/dhrubakumardolai/Desktop/Python Test/Rahul/Is in OB?/Target/latest ob/{Year}/{Month}/"
input_loc_ob_google = f"/home/dhrubakumardolai/Desktop/Python Test/Revisit File/Google Revisit/{Year}/{Month_n}/"
input_loc_ob_rwm = f"/home/dhrubakumardolai/Desktop/Python Test/Revisit File/OB/{Year}/{Month}/{Date_n_ob_no_change}/"

In [5]:
def fill_task(row2):
    if row2['Is in Google Daily File?'] == "Yes" and row2['CompletedSubTaskTypes'] is np.nan:
        return 0
    else:
        return row2['Task Count']

In [6]:
def Check_False(row3):
    if row3['Previous MID'] is np.nan:
        return np.nan
    elif row3['Previous MID'] != row3['MerchantExternalId']:
        return False
    elif row3['Previous MID'] == row3['MerchantExternalId']:
        return True
    else:
        return np.nan

In [7]:
def left_join_date_wise(left_table, left_date, right_table, right_date,
 common_col, merge_table_name, new_column_name, Date, Primary_key):
    merge_table_name = pd.DataFrame()
    for i in Date:
        t = left_table[left_table[left_date].isin([i])]
        m = right_table[right_table[right_date].isin([i])]
        t_merge = t.merge(m, on = common_col, how="left", indicator=True)
        t_merge.drop_duplicates(subset=[Primary_key], inplace=True)
        t_merge.rename(columns={'_merge':new_column_name}, inplace=True)
        t_merge[new_column_name].replace('left_only', "No", inplace=True)
        t_merge[new_column_name].replace('both', "Yes", inplace=True)
        merge_table_name = merge_table_name.append(t_merge)
    return merge_table_name

In [8]:
filelist = glob.glob(input_loc+"/*.csv", recursive=True)

In [9]:
target = pd.DataFrame()
for i in filelist:
    df = pd.read_csv(i)
    target = target.append(df)

In [10]:
target.head(1)

,City Name,Merchant ID,Merchant Name,Visit Count,Business Category,Owner Mobile No.,Follow Up Date,PIN Code,On Boarding Date,On Boarding Time,...,Revisit Expire Date,Source_MID,Hustle Task Completion Image,Data Tag,Unnamed: 33,sno,Dup_MerchantID,Report_Date,Revisit_Date,Dup
0,Bardhaman,9518165.0,KALIMATA MOBILE SERVICE CENTER,2.0,Service,9641025856,NaN,713502.0,Jun-27-2023,10:49:01,...,NaN,NaN,http://imagev1.fleetrobo.com/App_Images/FSTSIm...,RWM,NaN,100001,NaN,NaN,NaN,NaN


In [11]:
len(target)

97708

In [12]:
target_f = target[['City Name','Merchant ID','Merchant Name','Visit Count','Business Category','Owner Mobile No.','PIN Code','On Boarding Date','On Boarding Time',
'On-Boarded Employee Code','Visit By','Employee Code','Role Name','POD Name','POD ID','Employment Type','Revisit Date','Revisit Time','Team Lead Name',
'Remarks','Visit Status','Reason for not Accepting Payments using G-Pay','New Collateral Given : QR Code label','New Collateral Given : G-Pay Accepted Sticker',
'New Collateral Given : Dangler','QR Code String','Not Interested in Gpay Reasons','Created By','Created Name','Created Date','Merchant Ext ID',
'Is in Google Daily File?','CompletedSubTaskTypes','Task Count','Is in Source File?_x','Validity','Invalid Reason','Segment','Source_MID','Hustle Task Completion Image',
'Data Tag']].rename(columns={'Is in Source File?_x':'Is in Source File?', 'Source_MID':'Previous MID', 'Data Tag':'Source Data picekd from?', 
                                                 'Hustle Task Completion Image':'Hustle_Completion_Image'})

In [13]:
target_f2 = target_f[target_f['Validity'].isin(['INVALID', np.nan])]

In [14]:
target = target_f2

In [15]:
target['Primary_key'] = range(1,(len(target)+1))

In [16]:
filelist = glob.glob(input_loc_ob+"/*.csv", recursive=True)

In [17]:
ob_con = pd.DataFrame()
for i in filelist:
    df = pd.read_csv(i)
    ob_con = ob_con.append(df)

In [18]:
ob_con['VerificationTimestamp'] = pd.to_datetime(ob_con['VerificationTimestamp'])
ob_con['VerificationTimestamp'] = ob_con['VerificationTimestamp'].dt.strftime('%d-%b-%Y')

In [19]:
ob_con_f = ob_con[ob_con['VerificationTimestamp'].str.contains(f"{Month}")]

In [20]:
ob_con_f2 = ob_con_f[['MerchantExternalId', 'VerificationTimestamp']]

In [21]:
target.rename(columns={'Merchant Ext ID':'MerchantExternalId'}, inplace=True)

In [22]:
target['Revisit Date'] = pd.to_datetime(target['Revisit Date'])
target['Revisit Date'] = target['Revisit Date'].dt.strftime("%d-%b-%Y")
Date = target['Revisit Date'].unique()

In [23]:
target_2 = pd.DataFrame()
for i in Date:
    t = target[target['Revisit Date'].isin([i])]
    m = ob_con_f2[ob_con_f2['VerificationTimestamp'].isin([i])]
    t_merge = t.merge(m, on='MerchantExternalId', how="left", indicator=True)
    t_merge.drop_duplicates(subset=['Primary_key'], inplace=True)
    t_merge.rename(columns={'_merge':'Present in Google Invalid?_2'}, inplace=True)
    t_merge['Present in Google Invalid?_2'].replace('left_only', np.nan, inplace=True)
    t_merge['Present in Google Invalid?_2'].replace('both', "Yes", inplace=True)
    target_2 = target_2.append(t_merge)

# OB_Google_Invalid Mapping for Date and Validity

In [24]:
filelist = glob.glob(input_loc_ob_google+"**/*/onboardings/data-00000-of-00001.csv", recursive = True)

In [25]:
google_csv_ob = pd.DataFrame()

for file in filelist:
    if file.endswith(".csv"):
        df = pd.read_csv(file)
        google_csv_ob = google_csv_ob.append(df)

In [26]:
google_csv_ob['VerificationTimestamp'] = pd.to_datetime(google_csv_ob['VerificationTimestamp'])
google_csv_ob['VerificationTimestamp'] = google_csv_ob['VerificationTimestamp'].dt.strftime('%d-%b-%Y')

In [27]:
google_csv_ob_f = google_csv_ob[google_csv_ob['VerificationTimestamp'].str.contains(f"{Month}")]

In [28]:
google_csv_ob_f2 = google_csv_ob_f[['MerchantExternalId', 'VerificationTimestamp', 'Validity']].rename(
    columns={'Validity':'Validity_google_ob'})

In [29]:
target_3 = pd.DataFrame()
for i in Date:
    t = target_2[target_2['Revisit Date'].isin([i])]
    m = google_csv_ob_f2[google_csv_ob_f2['VerificationTimestamp'].isin([i])]
    t_merge = t.merge(m, on='MerchantExternalId', how="left", indicator=True)
    t_merge.drop_duplicates(subset=['Primary_key'], inplace=True)
    t_merge.rename(columns={'_merge':'Is_in_OB_Google_CSV_file'}, inplace=True)
    t_merge['Is_in_OB_Google_CSV_file'].replace('left_only', "No", inplace=True)
    t_merge['Is_in_OB_Google_CSV_file'].replace('both', "Yes", inplace=True)
    target_3 = target_3.append(t_merge)

In [30]:
target_3['VerificationTimestamp_x'].fillna(target_3['VerificationTimestamp_y'], inplace=True)
target_3.drop(['VerificationTimestamp_y'], axis=1, inplace=True)

In [31]:
target_3.rename(columns={'VerificationTimestamp_x':'VerificationTimestamp'}, inplace=True)

In [32]:
target_3.rename(columns={"VerificationTimestamp":'Valid_OB_Date'}, inplace=True)

In [33]:
target_3['Valid_OB_Date'].unique()

array([nan, '01-Jul-2023', '02-Jul-2023', '03-Jul-2023', '04-Jul-2023',
       '05-Jul-2023', '06-Jul-2023', '07-Jul-2023', '08-Jul-2023',
       '09-Jul-2023', '10-Jul-2023', '11-Jul-2023', '12-Jul-2023',
       '13-Jul-2023', '14-Jul-2023', '15-Jul-2023', '16-Jul-2023',
       '17-Jul-2023'], dtype=object)

In [34]:
#target_3["Onboarding Date from google invalid"].unique()

In [35]:
target_3.rename(columns={"Validity_google_ob_y":'Validity_google_ob'}, inplace=True)

In [36]:
target_3['Validity_google_ob'].fillna(target_3['Present in Google Invalid?_2'], inplace=True)
target_3['Validity_google_ob'].replace("Yes", "INVALID", inplace=True)

In [37]:
target_3['Validity_google_ob'].unique()

array([nan, ' INVALID', 'VALID ', 'INVALID'], dtype=object)

In [38]:
target_3['Validity_google_ob'] = target_3['Validity_google_ob'].str.replace(" ", "")

In [39]:
target_2 = target_3

In [40]:
target_2.rename(columns={'Present in Google Invalid?_2':'Present in Google Invalid?','Valid_OB_Date':'Onboarding Date from google invalid'}, inplace=True)

In [41]:
target_2['MerchantExternalId'].replace('Empty', np.nan, inplace=True)

In [42]:
target_2.sort_values(by=['Revisit Date'], inplace=True)
target_2['Dup_Check'] = target_2.groupby('MerchantExternalId')['MerchantExternalId'].rank(method='first')
target_2['Dup_Check'].replace(np.nan, 0, inplace=True)

In [43]:
target_2.rename(columns={'Dup_Check':'Dup Check MID'}, inplace=True)

In [44]:
target_2['Dup Check MID'].unique()

array([1., 2., 3.])

In [45]:
target_3 = target_2[~target_2['Revisit Date'].isin([np.nan])]

In [46]:
target_3['Task Count_2'] = target_3.apply(fill_task, axis=1)
target_3.drop(['Task Count'], axis=1, inplace=True)
target_3.rename(columns={'Task Count_2':'Task Count'}, inplace=True)

In [47]:
target_3['Invalid_PhonenumberLookup'] = target_3['Invalid Reason'].str.contains('Invalid_PhonenumberLookup', na=False)
target_3['Invalid_NotOnTargetlist'] = target_3['Invalid Reason'].str.contains('Invalid_NotOnTargetlist', na=False)

In [48]:
target_3['Validity_google_ob'].unique()

array([nan, 'INVALID', 'VALID'], dtype=object)

In [49]:
target_3['Validity_google_ob'] = target_3['Validity_google_ob'].str.replace(" ", "", regex=True)

In [50]:
#target_3['Present in Google Invalid?'].fillna(target_3['Validity_google_ob'], inplace=True)
#target_3['Present in Google Invalid?'].replace('INVALID', "Yes", inplace=True)
#target_3['Present in Google Invalid?'].replace('VALID', "Yes", inplace=True) 

# RWM OB Date and Validity Remarks

In [51]:
rwm_ob_validity = pd.read_csv(input_loc_ob_rwm+"Is Suspicious_Updated.csv")

In [52]:
rwm_ob_validity.head(1)

,Merchant ID,Is Suspicious,Final
0,9531548,No,Valid


In [53]:
rwm_ob_validity_f = rwm_ob_validity[['Merchant ID', 'Final']]

In [54]:
External_Gpay_fos = pd.read_csv(input_loc_ob_rwm+"External Gpay FOS Backend_Trail - Gpay Internal.csv")

In [55]:
External_Gpay_fos_f = External_Gpay_fos[['On Boarding Date', 'Owner Mob No', 'Merchant ID']]

In [56]:
External_Gpay_fos_f['On Boarding Date'] = pd.to_datetime(External_Gpay_fos_f['On Boarding Date'])
External_Gpay_fos_f['On Boarding Date'] = External_Gpay_fos_f['On Boarding Date'].dt.strftime("%d-%b-%Y")

In [57]:
External_Gpay_fos_f_merge = External_Gpay_fos_f.merge(rwm_ob_validity_f, on="Merchant ID", how="left")

In [58]:
External_Gpay_fos_f_merge.head(1)

,On Boarding Date,Owner Mob No,Merchant ID,Final
0,01-Jul-2023,BCR2DN4T53A4XX2V,9531548,Valid


In [59]:
External_Gpay_fos_f_merge_f = External_Gpay_fos_f_merge[['Owner Mob No', 'Final', 'On Boarding Date']].rename(
    columns={'On Boarding Date':'RWM OB Date', 'Owner Mob No':'MerchantExternalId'})

In [60]:
len(External_Gpay_fos_f_merge) - len(External_Gpay_fos_f_merge['Owner Mob No'])

0

# RWM Valid OB with Date

In [61]:
Date_for_mapping = target_3['Revisit Date'].unique()

In [62]:
target_4 = pd.DataFrame()
target_4 = left_join_date_wise(target_3, 'Revisit Date', External_Gpay_fos_f_merge_f, 'RWM OB Date',
 'MerchantExternalId', target_3, 'Is_in_OB_RWM_file', Date_for_mapping,'Primary_key')

In [63]:
target_4.head(1)

,City Name,Merchant ID,Merchant Name,Visit Count,Business Category,Owner Mobile No.,PIN Code,On Boarding Date,On Boarding Time,On-Boarded Employee Code,...,Present in Google Invalid?,Validity_google_ob,Is_in_OB_Google_CSV_file,Dup Check MID,Task Count,Invalid_PhonenumberLookup,Invalid_NotOnTargetlist,Final,RWM OB Date,Is_in_OB_RWM_file
0,Hingoli,9439962.0,SUNNY S%2FO VILAYATI SINGH,2.0,Stores & Shopping,8750305816,431513.0,Jun-01-2023,18:20:01,52084.0,...,NaN,NaN,No,1.0,1.0,True,False,NaN,NaN,No


In [64]:
target_4['Dynamic Source MID Check'] = target_4.apply(Check_False, axis=1)

In [65]:
def payout_remarks(row):
   if row['Dup Check MID'] > 1:
       return 'Duplicate Ext_ID'
   elif row['Dup Check MID'] == 0:
       return 'Not Found'
   elif row['Is in Google Daily File?'] == "No" and row['Present in Google Invalid?'] == "Yes" and row['Validity_google_ob'] == "INVALID" and row['Dynamic Source MID Check'] == False:
       return 'Possible -Re OB'
       
   elif row['Is in Google Daily File?'] == "No" and row['Present in Google Invalid?'] == "Yes" and row['Is in Source File?'] in ('No Match','No Match Dynamic'):
        return 'Not Found, Invalid_NotOnTargetlist'
   elif row['Final'] == "Valid":
       return 'Considered in Onboarding'
   elif row['Is in Google Daily File?'] == "No" and row['Present in Google Invalid?'] == "Yes" and row['Is in Source File?'] == 'No Match':
       return 'Not Found, Invalid_NotOnTargetlist'
   elif row['Is in Google Daily File?'] == "No" and row['Present in Google Invalid?'] != "Yes" and row['Dynamic Source MID Check'] != False:
       if row['Is in Source File?'] in ('Match','MN Match'):
           return 'Not Found'
       elif row['Is in Source File?'] in ('No Match','No Match Dynamic'):
           return 'Not Found, Invalid_NotOnTargetlist'
       else:
           return 'Not Found'
   elif row['Is in Google Daily File?'] == "No" and row['Present in Google Invalid?'] == np.nan and row['Is in Source File?'] in ('No Match','No Match Dynamic'):
       return 'Not Found, Invalid_NotOnTargetlist'
   elif row['Is in Google Daily File?'] == "No" and row['Source Data picekd from?'] == 'Form Data' and row['MerchantExternalId'] == np.nan:
       return 'Not Found'
   elif row['Is in Google Daily File?'] == "No" and row['Source Data picekd from?'] == 'RWM' and row['MerchantExternalId'] == np.nan:
       return 'Not Found'
   elif row['Is in Google Daily File?'] == "No" and row['Source Data picekd from?'] == 'RWM' and row['Is in Source File?'] == 'Match':
       return 'Not Found'
   elif row['Is in Google Daily File?'] == "No" and row['Source Data picekd from?'] == 'RWM' and row['Is in Source File?'] == 'MN Match':
       return 'Not Found'
   elif row['Is in Google Daily File?'] == "Yes" : 
       if row['Task Count'] < 3:
           if row['Invalid_PhonenumberLookup'] == True:
               return 'Invalid_PhonenumberLookup'
           else:
               return 'Invalid_IncompleteSubTasks'
       elif row['Task Count'] >= 3:
           if row['Is in Source File?'] == 'MN Match' and row['Validity'] == "INVALID":
               return "Invalid_NotOnTargetlist"
           elif row['Is in Source File?'] != 'MN Match' and row['Invalid_PhonenumberLookup'] == True:
               return "Invalid_PhonenumberLookup"
           elif row['Is in Source File?'] != 'MN Match' and row['Invalid_NotOnTargetlist'] == True:
               return "Invalid_NotOnTargetlist"
           else:
               if row['Validity'] == "VALID":
                    return "Valid"
               else:
                    return np.nan
   else:
       return "Not Found"



In [66]:
target_4['Payout_remarks_2'] = target_4.apply(payout_remarks, axis=1)

In [67]:
target_4['Payout_remarks_2'].value_counts()

Possible -Re OB                       5271
Invalid_NotOnTargetlist               1688
Invalid_PhonenumberLookup              682
Invalid_IncompleteSubTasks             672
Not Found, Invalid_NotOnTargetlist     608
Not Found                              589
Duplicate Ext_ID                        44
Name: Payout_remarks_2, dtype: int64

In [68]:
target_4.sort_values(by=['Primary_key'], inplace=True)

In [69]:
target_4['cluster'] = np.nan
target_4['Hustle Images'] = np.nan
target_4['Image Found'] = np.nan
target_4['ObjectivesDuringRevisit'] = np.nan

In [70]:
target_4['Image Found New'] = np.nan
target_4['Merchant Name Matched/ Unmatched'] = np.nan
target_4['SS Time Format'] = np.nan
target_4['SS Time Vs Revisit Time (15 mins)'] = np.nan
target_4['Invalid/ Valid (Time Matched)'] = np.nan
target_4['Assigned to'] = np.nan
target_4['Date'] = np.nan

In [ ]:
target_5 = target_4[['City Name',
'Merchant ID',
'Merchant Name',
'Owner Mobile No.',
'PIN Code',
'Visit By',
'Employee Code',
'Role Name',
'POD Name',
'POD ID',
'Revisit Date',
'Revisit Time',
'Team Lead Name',
'Remarks',
'QR Code String',
'MerchantExternalId',
'Is in Google Daily File?',
'CompletedSubTaskTypes',
'Task Count',
'Source Data picekd from?',
'cluster',
'Payout_remarks_2',
'Dup Check MID',
'Segment',
'ObjectivesDuringRevisit',
'Onboarding Date from google invalid',
'Present in Google Invalid?',
'Validity_google_ob',
'Dynamic Source MID Check',
'Validity',
'Invalid Reason',
'Previous MID',
'Is in Source File?',
'Image Found New',
'Merchant Name Matched/ Unmatched',
'SS Time Format',
'SS Time Vs Revisit Time (15 mins)',
'Hustle_Completion_Image',
'Invalid/ Valid (Time Matched)',
'Assigned to',
'Date',
'Image Found']].rename(columns={'Payout_remarks_2':'Final Remark'})

In [ ]:
target_5.to_csv(output_loc+'New_Current_Payout_Remarks.csv', index=False)

In [ ]:
target_5[target_5['Revisit Date'].isin(filter_range)].to_csv(output_loc+'Filter_Current_Payout_Remarks.csv', index=False)
target_5[target_5['Source Data picekd from?'].isin(filter_form)].to_csv(output_loc+'Form_Data_filter_Current_Payout_Remarks.csv', index=False)